In [24]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('/home/javier.mas/hyperband/hyperband')
import random
import pandas as pd
import numpy as np
import xgboost as xgb
from math import log
from sklearn.metrics import log_loss, cohen_kappa_score, confusion_matrix
from hyperband.hyperband import Hyperband
from hyperband.defs.xgb import get_params, try_params


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [23]:
!ls 

data	   Hyperband-test.ipynb  notebooks  Untitled.ipynb
hyperband  __init__.py		 tiago


In [9]:
data = pd.read_csv('data/HR_comma_sep.csv')
data = pd.get_dummies(data)
data_dict = dict()
feats = [col for col in data.columns if col != 'left']
data_dict['x_train'], data_dict['y_train'] = data.loc[int(len(data)*0.8):, feats], data.loc[int(len(data)*0.8):, 'left']
data_dict['x_test'], data_dict['y_test'] = data.loc[:int(len(data)*0.8), feats], data.loc[:int(len(data)*0.8), 'left']

In [10]:
data_dict['x_test']

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,promotion_last_5years,sales_IT,sales_RandD,sales_accounting,sales_hr,sales_management,sales_marketing,sales_product_mng,sales_sales,sales_support,sales_technical,salary_high,salary_low,salary_medium
0,0.38,0.53,2,157,3,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0
1,0.80,0.86,5,262,6,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1
2,0.11,0.88,7,272,4,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1
3,0.72,0.87,5,223,5,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0
4,0.37,0.52,2,159,3,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0
5,0.41,0.50,2,153,3,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0
6,0.10,0.77,6,247,4,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0
7,0.92,0.85,5,259,5,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0
8,0.89,1.00,5,224,5,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0
9,0.42,0.53,2,142,3,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0


In [11]:
hb = Hyperband(get_params, try_params)
results = hb.run(data=data_dict)


*** 81 configurations x 1.0 iterations each

1 | Thu Sep 28 00:13:23 2017 | lowest loss so far: inf (run -1)

n_estimators: 5
{'learning_rate': 0.10948073658267082,
 'max_depth': 2,
 'max_features': None,
 'min_samples_leaf': 5,
 'min_samples_split': 19,
 'subsample': 0.9773564962320919}

# training | log loss: 50.17%, AUC: 91.96%, accuracy: 91.17%
# testing  | log loss: 50.95%, AUC: 91.37%, accuracy: 88.22%

0 seconds.

2 | Thu Sep 28 00:13:23 2017 | lowest loss so far: 0.5095 (run 1)

n_estimators: 5
{'learning_rate': 0.058899244976344516,
 'max_depth': 10,
 'max_features': 'log2',
 'min_samples_leaf': 1,
 'min_samples_split': 11,
 'subsample': 0.8654569276546461}

# training | log loss: 49.09%, AUC: 99.73%, accuracy: 97.83%
# testing  | log loss: 52.74%, AUC: 98.57%, accuracy: 95.90%

0 seconds.

3 | Thu Sep 28 00:13:23 2017 | lowest loss so far: 0.5095 (run 1)

n_estimators: 5
{'learning_rate': 0.10448357968405217,
 'max_depth': 4,
 'max_features': 'sqrt',
 'min_samples_leaf': 4,


### Tic-tac-toe

In [25]:
data = pd.read_csv('data/tictac.csv', sep=';')

In [26]:
data = data.rename(columns={'1.5':'target'})

In [27]:
train_prop = 0.3
n_0 = data[data['target'] == 0].shape[0]
n_1 = data[data['target'] == 1].shape[0]
train_0 = random.sample(range(n_0), int(n_0*train_prop))
train_1 = random.sample(range(n_1), int(n_1*train_prop))
test_0 = [i for i in range(n_0) if i not in train_0]
test_1 = [i for i in range(n_1) if i not in train_1]
train = pd.concat((data[data['target'] == 0].iloc[train_0,:], data[data['target'] == 1].iloc[train_1,:]))
test = pd.concat((data[data['target'] == 0].iloc[test_0,:], data[data['target'] == 1].iloc[test_1,:]))

In [28]:
data_dict = {
    'x_train': train.drop('target', axis=1),
    'y_train': train['target'],
    'x_test': test.drop('target', axis=1),
    'y_test': test['target']
}

In [34]:
hb = Hyperband(get_params, try_params, max_iter=300, eta=3)
results = hb.run(data=data_dict)


*** 243 configurations x 1.2 iterations each

1 | Thu Oct 26 04:30:47 2017 | lowest loss so far: inf (run -1)

n_estimators: 6
{'base_score': 0.33567544817114814,
 'colsample_bylevel': 0.6287811345948335,
 'colsample_bytree': 0.9425677765694029,
 'max_depth': 8,
 'min_child_weight': 8,
 'reg_lambda': 8.667336846078264}

# training | log loss: 68.27%, AUC: 74.29%, accuracy: 53.50%
# testing  | log loss: 68.58%, AUC: 73.32%, accuracy: 52.46%

0 seconds.

2 | Thu Oct 26 04:30:47 2017 | lowest loss so far: 0.6858 (run 1)

n_estimators: 6
{'base_score': 0.21032533458460032,
 'colsample_bylevel': 0.5323532053238539,
 'colsample_bytree': 0.6176629250498963,
 'min_child_weight': 9}

# training | log loss: 72.99%, AUC: 73.42%, accuracy: 34.62%
# testing  | log loss: 74.12%, AUC: 63.39%, accuracy: 34.72%

0 seconds.

3 | Thu Oct 26 04:30:47 2017 | lowest loss so far: 0.6858 (run 1)

n_estimators: 6
{'colsample_bylevel': 0.7898455732075181,
 'max_depth': 7,
 'min_child_weight': 3,
 'reg_alpha': 

In [33]:
hb.results

[{'auc': 0.98000568326572213,
  'counter': 1,
  'iterations': 2.4,
  'log_loss': 0.50362370652875321,
  'loss': 0.50362370652875321,
  'params': {'base_score': 0.49261530391627795,
   'colsample_bylevel': 0.8598587930959023,
   'gamma': 0.3642883152833015,
   'learning_rate': 0.0860038674839635,
   'max_depth': 8,
   'reg_lambda': 7.645837501391092,
   'scale_pos_weight': 0.9269814218271035},
  'seconds': 0},
 {'auc': 0.84411194073725682,
  'counter': 2,
  'iterations': 2.4,
  'log_loss': 0.61388811580115032,
  'loss': 0.61388811580115032,
  'params': {'colsample_bylevel': 0.6292199627538462,
   'max_depth': 6,
   'scale_pos_weight': 0.25456032618956326,
   'subsample': 0.5221121193544558},
  'seconds': 0},
 {'auc': 0.78120896780136007,
  'counter': 3,
  'iterations': 2.4,
  'log_loss': 0.64036258677611946,
  'loss': 0.64036258677611946,
  'params': {'base_score': 0.4913911557274533,
   'colsample_bylevel': 0.9386799295359937,
   'colsample_bytree': 0.5731609493482342,
   'learning_rat